# Plato Tutorial

Plato is a python package built on top of [Theano](http://deeplearning.net/software/theano/) with two objectives:  
1) Simplify the use of Theano.  
2) Build a good libary of standard Deep Learning algorithms.

This tutorial takes you throught the Plato API.  It's useful but not necessary to have a basic knowledge of Theano to do this tutorial.

## 1. Symbolic functions.

In Plato, we have the concept of *symbolic functions*, which are function that take and return theano symbolic variables.  These functions can be compiled to *numeric functions* which take and return numpy arrays and python ints/floats.

In [3]:
from plato.core import symbolic

@symbolic
def add_two_numbers(x, y):
    return x+y

f = add_two_numbers.compile()
print '3+4=%s' % f(3, 4)

3+4=7


Internally, here is what happens: On the first (and in this case, only) call to add_two_numbers, Plato inspects the inputs (3, 4), looks at their type (both scalar integers in this case), and gets Theano to compile a symbolic expression that adds them together.  The equivalent code using just theano would be:

In [9]:
import theano
from theano.tensor import scalar

x = scalar(dtype = 'int32')
y = scalar(dtype = 'int32')
z = x+y

f = theano.function(inputs = [x, y], outputs = z)
print '3+4=%s' % f(3, 4)


3+4=7


/Users/peter/projects/argmaxlab/venv/src/theano/theano/tensor/var.py:128: UserWarning: Warning, Cannot compute test value: input 0 (<TensorType(int32, scalar)>) of Op Elemwise{add,no_inplace}(<TensorType(int32, scalar)>, <TensorType(int32, scalar)>) missing default value
  return theano.tensor.basic.add(self, other)
/Users/peter/projects/argmaxlab/venv/src/theano/theano/tensor/var.py:128: UserWarning: Warning, Cannot compute test value: input 1 (<TensorType(int32, scalar)>) of Op Elemwise{add,no_inplace}(<TensorType(int32, scalar)>, <TensorType(int32, scalar)>) missing default value
  return theano.tensor.basic.add(self, other)


Thus the first advantage of Plato is that it removes the need to create input variables and make sure their type matches the data that you're going to feed in.

## 2. Adding State

We are also able to create stateful functions.  Unlike Theano, we do not pass state-updates in the return value.  Instead, we call the function `add_update(shared_var, new_value)`.  The following example shows how to make a "function" with some internal state that updates on each call.

In [8]:
import theano
from plato.core import symbolic, add_update

@symbolic
def counter():
    count = theano.shared(0)  # Create a shared variable, initialized at zero, which stores the count.
    new_count = count+1
    add_update(count, new_count)
    return new_count

f = counter.compile()
print 'I can count to ten.  See: %s' % ([int(f()) for _ in xrange(10)])

f2 = counter.compile()
print 'I can too: %s' % ([int(f2()) for _ in xrange(10)])
# Note that we start from scratch when we compile the function a new time, 
# because the shared variable is initialized within the function call.  If we
# had declaired counter outside the function, the second count would run from 
# 11 to 20.

I can count to ten.  See: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
I can too: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## 3. Classes: Multiple functions sharing a variable.

We often have situations where we have a variable that is shared between two functions (e.g. in a classifier, the weights may be modified by the *train* function and used by the *predict* function).  We usually deal with this using classes.  As a simple example, lets take the [Collatz Conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture).  

In [7]:
from plato.core import symbolic, add_update
import theano

class Collatz:

    def __init__(self, initial_value):
        self.value = theano.shared(initial_value)
        
    @symbolic
    def divide_by_2(self):
        new_value = self.value/2
        add_update(self.value, new_value)
        return new_value
    
    @symbolic
    def multiply_by_3_and_add_one(self):
        new_value = self.value*3+1
        add_update(self.value, new_value)
        return new_value
    
c = Collatz(20)
div_fcn = c.divide_by_2.compile()
mul_fcn = c.multiply_by_3_and_add_one.compile()

value = c.value.get_value()
print 'Demonstrate Collatz conjecture for initial value of %s' % c.value.get_value()

while value != 1:
    value = div_fcn() if value % 2 == 0 else mul_fcn()
    print value

print 'Note that since the value is attached to the instance, it persists if functons are recompiled.'
new_div_fcn = c.divide_by_2.compile()
new_mul_fcn = c.multiply_by_3_and_add_one.compile()
for _ in xrange(6):
    value = new_div_fcn() if value % 2 == 0 else new_mul_fcn()
    print value

Demonstrate Collatz conjecture for initial value of 20
10
5
16
8
4
2
1
Note that since the value is attached to the instance, it persists if functons are recompiled.
4
2
1
4
2
1


## 4. Callable Classes

In python, classes can also act as functions, if they implement a `__call__` method.  This can be useful when you want to make parameterized functions.  Therefore Plato also allows you to decorate callable classes.  For example:

In [6]:
from plato.core import symbolic

@symbolic
class MultiplyBySomething:
    
    def __init__(self, what):
        self.what = what
        
    def __call__(self, x):
        return x*self.what
    
f = MultiplyBySomething(3).compile()

print '3*4=%s' % f(4)

3*4=12


## 5. Named Arguments

Unlike Theano, Plato allows you to pass inputs into compiled functions by name.  The only requirement is that you are consistent with their usage (if you call the function as `f(3, y=4)` the first, time, you cannot call it as `f(3, 4)` the next time, otherwise you will get an error.  See the following example:

In [5]:
from plato.core import symbolic

@symbolic
def add_and_div(x, y, z):
    return (x+y)/z

f = add_and_div.compile()
print '(2+4)/3 = %s' % f(x=4, y=2, z=3)
print '(1+3)/2 = %s' % f(z=2, y=3, x=1)

try:
    print 'Lets try again, but leave x as an unnamed arg...'
    f(2, y=4, z=3.)
except KeyError as e:
    print 'You were inconsistent - referenced x as a kwarg in the first call but not the second.'


(2+4)/3 = 2
(1+3)/2 = 2
Lets try again, but leave x as an unnamed arg...
You were inconsistent - referenced x as a kwarg in the first call but not the second.


## 6. Debugging

A big advantage of Plato is easier debugging.  There are two ways in which Plato helps you debug.

### 6A: Testing Initial Values

Theano allows you to add "test-values" to your symbolic variables ([see tutorial](http://deeplearning.net/software/theano/tutorial/debug_faq.html)).  This helps to catch shape-errors at compile-time instead of run-time, where it is difficult to find the line of code that caused them.  However, it can be a bit of extra work for the programmer, because they have to manually attach test values to their variables.  Fortunately, since Plato compiles your functions on the first pass, it can attach test-values "under the hood".

For example, lets look at a matrix multiplication, where we accidently get the shapes of our matrices wrong.  Since all inputs are given test values, we can easily track down the error - the traceback will lead back to the correct line.  This would not have been possible without test values, because the error would occur in the compiled code, which is no-longer linked to the source code.

Plato attaches the following properties to all symbolic variables:  
**var.ival** - The initial value of the variable (a numpy array or scalar)  
**var.ishape** - The initial shape of the variable  
**var.indim** - The initial number of dimensions of the variable  
**var.idtype** - The initial dtype of the variable  

In [4]:
import numpy as np
from plato.core import symbolic

@symbolic
def forward_pass(x, w):
    print 'x-shape: %s' % (x.ishape, )
    print 'w-shape: %s' % (w.ishape, )
    # Note that the above test-values only display on the first iteration.
    return x.dot(w)

f = forward_pass.compile()

try:
    # The following will cause an error (because second argument should have shape (4, 3))
    h = f(np.random.randn(5, 4), np.random.rand(3, 4))  
except ValueError as err:
    # If you do not catch the error, you get a stacktrace which points to the line at fault.
    print '%s: %s' % (err.__class__.__name__, err.message)


x-shape: (5, 4)
w-shape: (3, 4)
ValueError: shapes (5,4) and (3,4) not aligned: 4 (dim 1) != 3 (dim 0)


### 6B: Variable Traces

It can also be useful to print/plot traces of intermediate values.  Ordinarily in theano, this would require setting those variables as outputs, and restructuring code to peek at what would normally be an internal variables.  Plato does a bit of magic which allows you to print/plot/do anything with internal variables.  The following example illustrates this: 

In [3]:
import numpy as np
from plato.core import symbolic, tdbprint
import theano.tensor as tt
import theano

class Layer:
    
    def __init__(self, w):
        self.w = theano.shared(w)
        
    @symbolic
    def forward_pass(self, x):
        pre_sigmoid = x.dot(self.w)
        tdbprint(pre_sigmoid, name = 'Pre-Sigmoid Activation')  # Here we make a trace of an internal variable
        y = tt.nnet.sigmoid(pre_sigmoid)
        return y
    
n_samples = 1
n_in = 4
n_out = 3
    
layer = Layer(np.random.randn(n_in, n_out))
fwd_fcn = layer.forward_pass.compile()
for _ in xrange(3):
    y = fwd_fcn(np.random.randn(n_samples, n_in))
    print 'Post Sigmoid Activation: %s' % (y)

Pre-Sigmoid Activation: [[ 4.40823978 -3.22244603  2.0160825 ]]
Post Sigmoid Activation: [[ 0.98796989  0.03832972  0.88247532]]
Pre-Sigmoid Activation: [[-0.84914375  1.78727461  1.03616111]]
Post Sigmoid Activation: [[ 0.29961251  0.85659281  0.73810861]]
Pre-Sigmoid Activation: [[-0.31623953  1.34197376  1.74131986]]
Post Sigmoid Activation: [[ 0.42159248  0.79281434  0.85085463]]


You can also plot internal variables using the function `tdbplot` in `plato.tools.tdb_plotting`, but this tutorial does not cover it.

## 7. Enforcing Interfaces

In larger programs, it can be useful to enforce interfaces - that is, functions are required to obey a certain contract.  This allows function A to use function B without knowing in particular which function it is.  For instance, you may have some code that iterates through a dataset and trains a predictor, but doesn't necessarily know what kind of predictor it is - just that it has a *train* function that accepts inputs and targets, and returns updates.

For this reason, we have an extended set of decorators which enforce type-checking on inputs, outputs, and updates.  

**`@symbolic`** - No format requirements.  
**`@symbolic_simple`** - Returns a single output variable.  
**`@symbolic_multi`** - Returns a tuple of output tensors.  
**`@symbolic_stateless`** - Makes no state updates.  
**`@symbolic_updater`** - Returns nothing and produces at least one state update.  

To make custom function con you can decorate with **`@SymbolicFunction(input_format, output_format, update_format)`**, where each of the arguments is an `IFormat` object.  See `plato.core` for examples.

When function fail to obey the contract specified by their decorators, a `SymbolicFormatError` is raised.

For example:

In [2]:
from plato.core import symbolic_stateless, symbolic, SymbolicFormatError, add_update
import theano

@symbolic_stateless # Bad! We decorated with "symbolic_stateless", but we make a state update inside
def running_sum(x):
    shared_var = theano.shared(0)
    y = x + shared_var
    add_update(shared_var, y) 
    return y 

f = running_sum.compile()
print 'Trying to run incorrectly-decorated function...'
try: 
    f(3)
except SymbolicFormatError as err:
    print '  %s: %s' % (err.__class__.__name__, err.message)

print 'Lets try again with the correct format....'

@symbolic
def running_sum(x):
    shared_var = theano.shared(0)
    y = x + shared_var
    add_update(shared_var, y) 
    return y 

f = running_sum.compile()

print '  cumsum([1,2,3,4]) = %s' % ([int(f(i)) for i in xrange(1, 5)], )

Trying to run incorrectly-decorated function...
  SymbolicFormatError: Function <function running_sum at 0x107169ed8> should have created no state updates, but it created updates: [(<TensorType(int64, scalar)>, Elemwise{add,no_inplace}.0)]
Lets try again with the correct format....
  cumsum([1,2,3,4]) = [1, 3, 6, 10]


## 8. Fixed Arguments

When you use a numpy array on a theano symbolic function, it treats it as a constant.  We can use the **fixed_args** argument to **compile()** to partially-specify a function.  Theano will then compile the function with these arguments as fixed constants.  For example:


In [1]:
from plato.core import symbolic

@symbolic
def multiply(x, y):
    return x*y

f_mult_by_3 = multiply.compile(fixed_args = dict(x=3))

print '3*2 = %s' % f_mult_by_3(y=2)
print '3*5 = %s' % f_mult_by_3(y=5)


3*2 = 6
3*5 = 15


## 9. Done.

Congratulations, you made it through the Plato tutorial.